In [ ]:
%%capture
# Install the latest version of attk (if necessary)
#!pip install -U git+git://github.com/hipstas/audio-tagging-toolkit.git

In [1]:
import attk
import os
import numpy as np
import timeit
import random
from sklearn.externals import joblib
from numpy import ma
from aubio import source, pitch
import librosa
from IPython.display import display, Audio
import subprocess
import unicodecsv
import urllib2
import csv


os.chdir('/sharedfolder/GitHub/sida/')

!mkdir /sharedfolder/GitHub/sida/___training_audio

os.chdir('/sharedfolder/GitHub/sida/___training_audio')

## Uncomment lines below to download audio files for training

#!wget http://www.stephenmclaughlin.net/HILT/audio_corpora/NPR_Fresh_Air_diarized.zip
#!unzip NPR_Fresh_Air_diarized.zip

mkdir: cannot create directory '/sharedfolder/GitHub/sida/___training_audio': File exists


In [ ]:
## Loading label data CSV as a list of lists

## Random 1-second labels
#csv_url = "https://raw.githubusercontent.com/hipstas/aapb-labels/master/Terry_Gross/Terry_Gross_labels.csv"

## Labels done by hand in Sonic Visualiser
csv_url = 'https://raw.githubusercontent.com/hipstas/aapb-labels/master/Terry_Gross/Terry_Gross_corrected_SV.csv'

csv_string = urllib2.urlopen(csv_url)

train_table = []

csv_reader = unicodecsv.reader(csv_string)

for row in csv_reader:
        train_table.append(row)

train_table[:10]+['...']

In [ ]:
## Removing header row if present

if 'Media file basename' in train_table[0]:
    train_table = train_table[1:]

random.shuffle(train_table)

In [ ]:
%%capture

## Excerpting WAV clips corresponding to labels

training_audio_pathname = "NPR_Fresh_Air_diarized"
out_dir = '_classes_SV_' + training_audio_pathname


for row in train_table:
    try:
        basename , start, duration, class_name, labeled_by = row  ## Assigning values in row to variables
        filename = str(basename + '.mp3')
        start = float(start)
        end = float(start) + float(duration)
        out_pathname = str(os.path.join(out_dir, class_name.replace(' ','_')))
        try: 
            subprocess.call(['mkdir', '-p', out_pathname])
        except:
            pass
        attk.subclip(os.path.join(training_audio_pathname, filename), float(start), end, out_pathname) ## <- attk
    except Exception as e: 
        print(row)
        print(e)



In [ ]:
################################################

In [ ]:

def extract_pairs(media_path,vowel_time_ranges):
    snd = AudioFileClip.AudioFileClip(media_path)
    file_duration = attk.duration(media_path)
    for pair in vowel_time_ranges:
        start, end = pair
        start = float(start)
        end = float(end)
        if end-start >= 0.1:  ## Ignore clips shorter than 0.1 second
            basename = media_path.split('/')[-1][:-4]
            out_filename = basename+'__'+str(round(start, 4))+'_'+str(round(end, 4))+'.wav'
            snd.subclip(start, end).write_audiofile(os.path.join('_vowel_clips',out_filename))


def batch_extract_vowels(media_dir):

    starting_location = os.getcwd()
    
    os.chdir(media_dir)

    bin_2048_to_sec_constant = 0.046439909297052155
    
    try: os.mkdir('_vowel_clips')
    except: pass

    filenames=[item for item in os.listdir('./') if item[-4:].lower() in ('.mp3','.wav','.mp4')]

    tic=timeit.default_timer()

    for filename in filenames[::-1]:
        try:
            vowel_bools = attk.get_vowel_segments(filename)

            vowel_bin_ranges = attk.labels_to_ranges(vowel_bools, label=True)

            vowel_time_ranges = [(s*bin_2048_to_sec_constant, e*bin_2048_to_sec_constant) for s, e in vowel_bin_ranges]
            
            extract_pairs(filename,vowel_time_ranges)

        except: print("ERROR: "+filename)

    print("Time elapsed: "+str(timeit.default_timer() - tic))

    os.chdir(starting_location)


In [ ]:
%%capture

os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_SV_NPR_Fresh_Air_diarized')

#for class_dir_name in [item for item in os.listdir('./') if os.path.isdir(item)]:
#    batch_extract_vowels(class_dir_name)

batch_extract_vowels('Terry_Gross')
#batch_extract_vowels('Background_Speaker')

In [ ]:
%%capture

os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_NPR_Fresh_Air_diarized')

#for class_dir_name in [item for item in os.listdir('./') if os.path.isdir(item)]:
#    batch_extract_vowels(class_dir_name)

batch_extract_vowels('Terry_Gross')
batch_extract_vowels('Background_Speaker')

In [ ]:
%%capture

os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_ubm_clips_final_July_2017')

#batch_extract_vowels('Female')
batch_extract_vowels('Male')


In [ ]:
## Extracting features
tic=timeit.default_timer()

os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_SV_NPR_Fresh_Air_diarized/')


dir_names = [item for item in os.listdir('./') if os.path.isdir(item)]

for dir_name in dir_names:
    
    os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_SV_NPR_Fresh_Air_diarized/' + dir_name + '/_vowel_clips')
    
    try: os.mkdir('../_vowel_mfccs_and_deltas')
    except: pass
    
    for filename in [item for item in os.listdir('./') if item.lower()[-4:] == '.wav']:
        csv_out_path = '../_vowel_mfccs_and_deltas/' + filename[:-4] + '.mfcc.csv'
        if not os.path.isfile(csv_out_path):
            try:
                mfccs = attk.get_mfccs_and_deltas(filename)
                with open(csv_out_path, 'w') as fo:
                    csv_writer = csv.writer(fo)
                    csv_writer.writerows(mfccs)  
            except:
                "ERROR on " + filename


print(timeit.default_timer() - tic)

tic=timeit.default_timer()

##


In [ ]:
## Extracting features
tic=timeit.default_timer()

os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_NPR_Fresh_Air_diarized/')

dir_names = [item for item in os.listdir('./') if os.path.isdir(item)]

for dir_name in dir_names:
    
    try:
        os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_NPR_Fresh_Air_diarized/' + dir_name + '/_vowel_clips')
    
        try: os.mkdir('../_vowel_mfccs_and_deltas')
        except: pass
    
        csv_out_path = '../_vowel_mfccs_and_deltas/' + filename[:-4] + '.mfcc.csv'
        if not os.path.isfile(csv_out_path):
            try:
                mfccs = attk.get_mfccs_and_deltas(filename)
                with open(csv_out_path, 'w') as fo:
                    csv_writer = csv.writer(fo)
                    csv_writer.writerows(mfccs)  
            except:
                "ERROR on " + filename
                
    except: pass

print(timeit.default_timer() - tic)

tic=timeit.default_timer()

##


In [ ]:
## Extracting features
tic=timeit.default_timer()

os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_ubm_clips_final_July_2017')

dir_names = [item for item in os.listdir('./') if os.path.isdir(item)]

for dir_name in dir_names:
    
    try:
        os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_ubm_clips_final_July_2017/' + dir_name + '/_vowel_clips')
    
        try: os.mkdir('../_vowel_mfccs_and_deltas')
        except: pass
    
        for filename in [item for item in os.listdir('./') if item.lower()[-4:] == '.wav']:
            mfccs = attk.get_mfccs_and_deltas(filename)
            with open('../_vowel_mfccs_and_deltas/' + filename[:-4] + '.mfcc.csv', 'w') as fo:
                csv_writer = csv.writer(fo)
                csv_writer.writerows(mfccs)  
    except: pass

print(timeit.default_timer() - tic)

tic=timeit.default_timer()

##


In [ ]:

os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_NPR_Fresh_Air_diarized/Terry_Gross/_vowel_mfccs_and_deltas')

gross_features = []

for filename in os.listdir('./'):
    with open(filename) as fi:
        csv_reader = csv.reader(fi)
        for row in csv_reader:
            gross_features.append([float(item) for item in row])

print(len(gross_features))


os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_SV_NPR_Fresh_Air_diarized/Terry_Gross/_vowel_mfccs_and_deltas')

gross_sv_features = []

for filename in os.listdir('./'):
    with open(filename) as fi:
        csv_reader = csv.reader(fi)
        for row in csv_reader:
            gross_sv_features.append([float(item) for item in row])

print(len(gross_sv_features))

In [ ]:
import csv

os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_NPR_Fresh_Air_diarized/Background_Speaker/_vowel_mfccs_and_deltas')

fa_ubm_features = []

for filename in os.listdir('./'):
    with open(filename) as fi:
        csv_reader = csv.reader(fi)
        for row in csv_reader:
            fa_ubm_features.append([float(item) for item in row])

print(len(fa_ubm_features))



In [ ]:
import csv

os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_ubm_clips_final_July_2017/Male/_vowel_mfccs_and_deltas')

m_ubm_features = []

for filename in os.listdir('./'):
    with open(filename) as fi:
        csv_reader = csv.reader(fi)
        for row in csv_reader:
            m_ubm_features.append([float(item) for item in row])

print(len(m_ubm_features))


os.chdir('/sharedfolder/GitHub/sida/___training_audio/_classes_ubm_clips_final_July_2017/Female/_vowel_mfccs_and_deltas')

f_ubm_features = []

for filename in os.listdir('./'):
    with open(filename) as fi:
        csv_reader = csv.reader(fi)
        for row in csv_reader:
            f_ubm_features.append([float(item) for item in row])

print(len(f_ubm_features))

In [2]:
os.chdir('/sharedfolder/GitHub/sida/___training_audio/')

In [ ]:
## Combining feature sets

speaker_1_mfccs = gross_features #+ random.sample(gross_sv_features, 1000)
bg_mfccs = fa_ubm_features + m_ubm_features + f_ubm_features

#speaker_1_mfccs = terry_gross_mfccs[::-1]
#bg_mfccs = ubm_mfccs[::-1]

random.shuffle(speaker_1_mfccs)
random.shuffle(bg_mfccs)

print(len(speaker_1_mfccs))
print(len(ubm_mfccs))

In [ ]:
## Decision tree

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import ExtraTreesClassifier

tic=timeit.default_timer()

X = speaker_1_mfccs[:-len(speaker_1_mfccs)/10] + ubm_mfccs[:-len(ubm_mfccs)/10]
y = [0]*len(speaker_1_mfccs[:-len(speaker_1_mfccs)/10]) + [1]*len(ubm_mfccs[:-len(ubm_mfccs)/10])

X_test = speaker_1_mfccs[-len(speaker_1_mfccs)/10:] + ubm_mfccs[-len(ubm_mfccs)/10:]
y_test = [0]*len(speaker_1_mfccs[-len(speaker_1_mfccs)/10:]) + [1]*len(ubm_mfccs[-len(ubm_mfccs)/10:])

#classifier = ExtraTreesClassifier().fit(X, y)
classifier = MLPClassifier().fit(X, y)

## Saving trained model
joblib.dump(classifier,'gross_vowels_mlpc_2048.pkl')
classifier = joblib.load('gross_vowels_mlpc_2048.pkl')

print(timeit.default_timer() - tic)

In [ ]:
classifier.score(X_test,y_test)

In [3]:
classifier = joblib.load('gross_vowels_mlpc_2048.pkl')



In [ ]:
## Combining feature sets

speaker_1_mfccs = gross_features #+ random.sample(gross_sv_features, 20000)
bg_mfccs = fa_ubm_features + m_ubm_features + f_ubm_features

#speaker_1_mfccs = terry_gross_mfccs[::-1]
#bg_mfccs = ubm_mfccs[::-1]

#random.shuffle(speaker_1_mfccs)
#random.shuffle(bg_mfccs)

print(len(speaker_1_mfccs))
print(len(ubm_mfccs))

In [ ]:
len(gross_sv_features)

In [ ]:
### KERAS

import pandas as pd

speaker_1_mfccs_df = pd.DataFrame(speaker_1_mfccs[:-500])

speaker_1_mfccs_df['Class Label'] = 1.0   ## Terry Gross


bg_mfccs_df = pd.DataFrame(bg_mfccs[:-500])

bg_mfccs_df['Class Label'] = 0.0    ## Background speaker


training_data_df = pd.concat([speaker_1_mfccs_df, bg_mfccs_df])

training_data_df.head()  



## Test fold
speaker_1_mfccs_df_test = pd.DataFrame(speaker_1_mfccs[-500:])
speaker_1_mfccs_df_test['Class Label'] = 1.0   ## Terry Gross

bg_mfccs_df_test = pd.DataFrame(bg_mfccs[-500:])
bg_mfccs_df_test['Class Label'] = 0.0    ## Background speaker

test_data_df = pd.concat([speaker_1_mfccs_df_test, bg_mfccs_df_test])

In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Data needs to be scaled to a small range like 0 to 1 for the neural
# network to work well.
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale both the training inputs and outputs
scaled_training = scaler.fit_transform(training_data_df)
scaled_testing = scaler.transform(test_data_df)

# Print out the adjustment that the scaler applied to the total_earnings column of data
#######print("Note: total_earnings values were scaled by multiplying by {:.10f} and adding {:.6f}".format(scaler.scale_[8], scaler.min_[8]))

# Create new pandas DataFrame objects from the scaled data
scaled_training_df = pd.DataFrame(scaled_training, columns=training_data_df.columns.values)
scaled_testing_df = pd.DataFrame(scaled_testing, columns=test_data_df.columns.values)

scaled_testing_df.head()


# Save scaled dataframes to new CSV files
scaled_training_df.to_csv("terry_gross_vs_ubm_train_scaled.csv", index=False)
scaled_testing_df.to_csv("terry_gross_vs_ubm_test_scaled.csv", index=False)

In [ ]:
## tic=timeit.default_timer()

import pandas as pd
from keras.models import Sequential
from keras.layers import *

training_data_df = pd.read_csv("terry_gross_vs_ubm_train_scaled.csv")

X = training_data_df.drop('Class Label', axis=1).values
Y = training_data_df[['Class Label']].values

# Define the model
model = Sequential()
model.add(Dense(200, input_dim=38, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(81, activation='relu'))
model.add(Dense(42, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss="mean_squared_error", optimizer="adam")


print(timeit.default_timer() - tic)

In [ ]:

# Train the model
model.fit(
    X,
    Y,
    epochs=50,
    shuffle=True,
    verbose=2
)

# Load the separate test data set
test_data_df = pd.read_csv("terry_gross_vs_ubm_test_scaled.csv")

X_test = test_data_df.drop('Class Label', axis=1).values
Y_test = test_data_df[['Class Label']].values

test_error_rate = model.evaluate(X_test, Y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: " + str(test_error_rate))

# Save the model to disk
model.save("trained_model.h5")
print("Model saved to disk.")




In [ ]:
test_error_rate

In [ ]:

def classify_clip(clip_pathname):
    mfccs = attk.get_mfccs_and_deltas(clip_pathname)
    results = classifier.predict(mfccs)  ## Predicting new observation
    vowel_results=[]
    vowel_bools = get_vowel_segments(clip_pathname)[::2]

    for i in range(len(results)):
        if vowel_bools[i]==True:
            vowel_results.append(results[i])

    return np.mean(vowel_results) ## Vowels only


In [ ]:
## Loading pre-trained model

#from sklearn.ensemble import RandomForestClassifier

#random_forest=joblib.load('pesca_vowels_random_forest_2048.pkl')

In [ ]:
## Classifying short clips


tic=timeit.default_timer()

filename = random.choice(os.listdir('unseen/'))
test_pathname = 'unseen/'+filename
test_mfccs=attk.get_mfccs_and_deltas(test_pathname)

print(test_pathname)

results = classifier.predict(test_mfccs)  ## Predicting new observation

print(results)


vowel_results=[]

vowel_bools = get_vowel_segments(test_pathname)[::2]

for i in range(len(results)):
    if vowel_bools[i]==True:
        vowel_results.append(results[i])

display(Audio(test_pathname))


print("All: "+str(np.mean(results)))
print("Vowels only: "+str(np.mean(vowel_results)))

#print("Time elapsed: "+str(timeit.default_timer() - tic))

In [ ]:
len(get_vowel_segments(test_pathname))

In [ ]:
!du /tmp/temp_clip.wav

In [ ]:


## Classifying a long audio file


tic=timeit.default_timer()


media_path = "/sharedfolder/fa_eval/FA_Author_Tom_Perrotta.mp3"


snd = AudioFileClip.AudioFileClip(media_path)

classifications=[]

for i in range(int(attk.duration(media_path))):
#    try:
        snd.subclip(i,i+1).write_audiofile('/tmp/temp_clip.wav')
        classifications.append(classify_clip('/tmp/temp_clip.wav'))
#    except: print('missed one')

classifications

In [ ]:
# Writing classification output to CSV

counter=0

class_0_secs=[]
class_1_secs=[]

i=0

for classification in attk.smooth(np.array(classifications)):
    if classification < 0.34:
        class_0_secs.append(i)
    if classification > 0.38:
        class_1_secs.append(i)
    i+=1


counter=0

csv_path=media_path[:-4]+'_extratrees2048_labels.csv'

with open(csv_path,'w') as fo:
    for pair in seconds_list_to_ranges(class_0_secs):
        fo.write(str(float(pair[0]))+','+str(float(pair[1]))+',Terry Gross\n')
    for pair in seconds_list_to_ranges(class_1_secs):
        fo.write(str(float(pair[0]))+','+str(float(pair[1]))+',Background\n')


print(timeit.default_timer() - tic)